In [2]:
from glob import glob
import os

files = glob('*.pdf')
files.extend(glob('*/*.pdf'))
files = sorted(list(set(files)))
files = [f for f in files if (os.path.getsize(f) / 1e6) > 2]
files

['1 Tahun-1-Bahasa-Inggeris-SJK.pdf',
 '1 Tahun-1-Bahasa-Melayu-SJK.pdf',
 '2 Tahun-1-Bahasa-Cina-SJKC.pdf',
 '2 Tahun-1-Matematik-Jilid-2-SJKC.pdf',
 '2 Tahun-1-Pendidikan-Jasmani-dan-Pendidikan-Kesihatan-SJKC.pdf',
 '2 Tahun-1-Pendidikan-Kesenian-SJKC.pdf',
 '2 Tahun-1-Pendidikan-Moral-SJKC.pdf',
 '2 Tahun-1-Sains-SJKC.pdf',
 '3 Tahun-1-Bahasa-Tamil-SJKT.pdf',
 '3 Tahun-1-Matematik-Jilid-1-SJKT.pdf',
 '3 Tahun-1-Matematik-Jilid-2-SJKT.pdf',
 '3 Tahun-1-Sains-SJKT.pdf',
 'Adab Wa Balaghah_Tingkatan_4.pdf',
 'Adab Wa Balaghah_Tingkatan_5_.pdf',
 "Al-Lughah Al-Arabiah Al-Mu'asirah_Tingkatan_5.pdf",
 "Al-Lughah Al-Arabiah Al-Mua'siarah Tingkatan 4.pdf",
 'Al-Syariah_Tingkatan 4.pdf',
 'Asas Kelestarian Tingkatan 4.pdf',
 'Asas Kelestarian_Tingkatan_5.pdf',
 'Asas Komputer Tingkatan 1.pdf',
 'Asas Sains Komputer Tingkatan 3.pdf',
 'Asas Sains Komputer Tingkatan_2.pdf',
 'B Arab.pdf',
 'B Iban.pdf',
 'B Kasazandusun.pdf',
 'B Semai.pdf',
 'B Tamil.pdf',
 'BA Tahun 1.pdf',
 'BAHASA MELAYU T

In [14]:
# !zip -r buku-teks.zip buku-teks

In [22]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup
from glob import glob

In [28]:
TIKA_HOST = 'http://localhost:9998'

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(file, minlen=2):
    raw_xml = parser.from_file(file, TIKA_HOST, xmlContent=True, requestOptions = {'timeout': 3600})
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass
    
    return t

In [29]:
with open('buku-teks.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            t = '\n'.join(parse_tika(f))
            data = {
                'file': f,
                'text': t,
            }
            fopen.write(f'{json.dumps(data)}\n')
        except Exception as e:
            print(e)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 208/208 [13:43<00:00,  3.96s/it]
